**DEATH PREDICT**

*Predicting heart failure in hospital ICU (Intensive car Unit)*

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Read CSV file into DataFrame df
df = pd.read_csv('/kaggle/input/in-hospital-mortality-prediction/data01.csv', index_col=0)
# Show dataframe
df

In [4]:
# Average Age of non-surviving patients 
deads = df[df.outcome == 1]
print(np.mean(deads.age))

In [5]:
# Visualization of Heart Failures
'''
fig, ax = plt.subplots()
dead_count = df['outcome'].value_counts() 
dead_count.plot(ax=ax, kind='bar')
'''

In [6]:
# Visualization of Heart Failures

plt.figure(figsize=(16,8))
plt.title('Frequency of Heart Failures')
plt.hist(df['outcome'])
plt.xlabel('Heart Failure (0 = No) (1 = Yes)', fontsize=18)
plt.ylabel('Number of patients', fontsize=18)
plt.show()

In [7]:
# prepare data for training
# remove outcome from the training

df1 = df.drop(columns=['outcome', 'ID'])
nmp = df1.to_numpy()
nmp

In [8]:
# get the number of rows to train the model on
training_data_len = math.ceil(len(nmp) * 0.8)
print(nmp.shape)
print(training_data_len)


In [9]:
# data exemple for 1 column (age)
data = nmp[:,0].reshape(-1,1)
# data for all columns
data = nmp

In [10]:
#scale the data ( convert value between 0 and 1)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)
scaled_data

In [11]:
# create the training dataset
ground_truth = df['outcome'].to_numpy()
#split the data into x_train and y_train data sets
x_train = scaled_data[:training_data_len, :]
y_train = ground_truth[:training_data_len]

#print(x_train)
#print(y_train)

In [13]:
# reshape the data
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
y_train = y_train.reshape(y_train.shape[0],1)

In [15]:
# build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [16]:
# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [17]:
#train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)